In [1]:
import MySQLdb
import pandas as pd
import numpy as np

db = MySQLdb.connect("localhost","root","","recommend")
user_cursor = db.cursor()
user_cursor.execute('SELECT user FROM view GROUP BY user HAVING (COUNT(user) > 20)')
user_result = user_cursor.fetchall()
user_list = []
for user in user_result:
    user_list.append(user[0])

In [2]:
print len(user_list)

85114


In [3]:
count = 0
user_count = len(user_list)
for user in user_list:
    count += 1
    if count%1000 == 0:
        userTuple= tuple(user_list[count-1000:count])
        sql = 'SELECT user, product_oid FROM view WHERE user IN' + str(userTuple)
        if count == 1000:
            df = pd.read_sql(sql, con=db)
        else:
            df_add = pd.read_sql(sql, con=db)
            df = pd.concat([df, df_add], ignore_index=True)
    elif count == user_count:
        userTuple= tuple(user_list[user_count-(count%1000)::])
        sql = 'SELECT user, product_oid FROM view WHERE user IN' + str(userTuple)
        df_add = pd.read_sql(sql, con=db)
        df = pd.concat([df, df_add], ignore_index=True)
           
df.head()

,user,product_oid
0,003e67b1b1e8259dadc124eebf15d659,6615
1,000e8afa513ca27d7ce2c91d93d59e2e,2311
2,000e8afa513ca27d7ce2c91d93d59e2e,2311
3,000e8afa513ca27d7ce2c91d93d59e2e,2311
4,000e8afa513ca27d7ce2c91d93d59e2e,2311


In [4]:
print len(df)

3636994


In [5]:
df.duplicated()

0          False
1          False
2           True
3           True
4           True
5           True
6           True
7           True
8           True
9          False
10         False
11          True
12         False
13         False
14         False
15         False
16         False
17          True
18          True
19         False
20         False
21         False
22         False
23         False
24         False
25         False
26         False
27         False
28          True
29         False
           ...  
3636964    False
3636965     True
3636966    False
3636967    False
3636968    False
3636969    False
3636970    False
3636971     True
3636972     True
3636973    False
3636974     True
3636975     True
3636976    False
3636977    False
3636978     True
3636979    False
3636980     True
3636981     True
3636982     True
3636983     True
3636984     True
3636985     True
3636986     True
3636987     True
3636988     True
3636989    False
3636990     True
3636991     Tr

In [6]:
df = df.drop_duplicates()
print len(df)

1566451


In [7]:
df['rating'] = 1
df.head()

,user,product_oid,rating
0,003e67b1b1e8259dadc124eebf15d659,6615,1
1,000e8afa513ca27d7ce2c91d93d59e2e,2311,1
9,003e67b1b1e8259dadc124eebf15d659,7444,1
10,003e67b1b1e8259dadc124eebf15d659,8185,1
12,003e67b1b1e8259dadc124eebf15d659,9914,1


In [8]:
productRatings = df.pivot_table(index=['user'],columns=['product_oid'],values='rating',aggfunc=lambda x: len(x.unique()),fill_value=0)
productRatings.shape

(85114, 5613)

In [9]:
productRatings.head()

product_oid,7,8,10,11,12,15,17,18,19,20,...,12083,12084,12085,12090,12094,12096,12098,12101,12102,12103
user,,,,,,,,,,,,,,,,,,,,,
000054b655862da1c03da11e22b3fd65,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0000c66a1046d6fd8945c07a7826bdf7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0001b8598a4619aaf1c1ff17b409c541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0001c93afae06d1fd35dc66fa3c042fd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000203e8ba594ad45e0139007cd290e9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
from sklearn.metrics.pairwise import pairwise_distances
item_similarity = 1 - pairwise_distances(productRatings.T, metric = "cosine")
item_similarity.shape

(5613, 5613)

In [11]:
item_similarity

array([[ 1.        ,  0.12538902,  0.0484249 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.12538902,  1.        ,  0.05840639, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.0484249 ,  0.05840639,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         1.        ,  1.        ]])

In [12]:
productRatings_pred = pd.DataFrame( 
    data=item_similarity[:,:],    
    index=productRatings.columns,    
    columns=productRatings.columns
) 
productRatings_pred.head()

product_oid,7,8,10,11,12,15,17,18,19,20,...,12083,12084,12085,12090,12094,12096,12098,12101,12102,12103
product_oid,,,,,,,,,,,,,,,,,,,,,
7,1.000000,0.125389,0.048425,0.225713,0.042429,0.275315,0.366120,0.008849,0.014183,0.005093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.125389,1.000000,0.058406,0.312998,0.032898,0.221841,0.202155,0.004574,0.009775,0.008775,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.048425,0.058406,1.000000,0.067227,0.034586,0.052806,0.047480,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.225713,0.312998,0.067227,1.000000,0.037202,0.260923,0.262650,0.011638,0.016581,0.011163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.042429,0.032898,0.034586,0.037202,1.000000,0.034700,0.017829,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
similar_products = pd.DataFrame(index=productRatings_pred.columns,columns=range(1,21))
for i in range(0,len(productRatings_pred.columns)): 
    similar_products.iloc[i,:20] = productRatings_pred.iloc[0:,i].sort_values(ascending=False)[:20].index

In [14]:
similar_products.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
product_oid,,,,,,,,,,,,,,,,,,,,
7,7,17,7347,15,11298,11300,11012,11,10591,11873,11464,9922,3145,9703,9926,11977,10319,10380,10208,9705
8,8,3145,11,4242,10380,8962,4587,10319,2322,4507,8641,9235,5896,9705,4812,4225,15,5043,4038,10591
10,10,3104,420,6140,5792,8297,10083,3225,10591,9926,9922,4242,4812,7347,6691,6136,6158,8962,9705,4587
11,11,8,8962,10380,4242,4587,9926,4225,10319,2322,4812,9235,17,15,9922,5043,4038,5896,7347,8641
12,12,4590,10083,3104,2373,3225,10073,6732,5792,6140,8297,6158,5072,6136,8450,7104,9892,9706,3913,8115


In [15]:
prod_oid = 10332
for i in range(20):
    print similar_products.loc[prod_oid][i+1], productRatings_pred.loc[prod_oid][similar_products.loc[prod_oid][i+1]]

10332 1.0
2989 0.439641112936
7423 0.43183130754
7452 0.331998443519
2881 0.274969616923
2878 0.241125222157
2930 0.236097532414
9675 0.228248742713
2768 0.223794472732
2612 0.212082859032
2603 0.193282426085
4052 0.190047322741
2914 0.18285578239
2948 0.175144418483
1922 0.172364412747
10780 0.17186482585
2764 0.165725187341
3530 0.16564383692
3014 0.163584136391
2358 0.15217046966
